## Desafío Caja los Andes

Hecho por: Felipe Araya Villela

### Resumen

En este notebook se realiza un modelo de clasifiación binaria utilizando regresión logística. En primer lugar, se explora el dataset otorgado por CLA, obteniendo que es un dataset con 2000 datos y 9 columnas, siendo una de estas el target, el cual está desbalanceado. Posteriormente se preprocesan los datos y se corre el modelo, obteniendo un F1-Score de 0.94 para los target igual a 1 y un F1-Score de 0.09 para los target igual a 0. Finalmente, se calcula el área bajo la curva ROC (AUC) para los conjuntos de entrenamiento y prueba, teniendo como resultados 0.90 y 0.71 respectivamente, indicando un posible sobreajuste en los datos.

### Carga de Librerías

In [94]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

### Carga de Datos

In [80]:
data = pd.read_csv('dataset_Caso_1.csv')

### EDA

**Comentarios Generales EDA:** Se cuenta con una base de datos de 2000 filas y 9 columnas, se puede observar que no cuenta con datos nulos y que dos de sus columnas son categóricas. Además, se pudo observar un gran desbalanceo en el target, encontrando que solo el 1,1% de los datos tienen un target igual a 1 (22 valores en total).

In [81]:
data.head()

,x1,x2,x3,x4,x5,x6,x7,x8,target
0,50.1341,-326.0000,SAT,MZBER,0,0,6,-6.5,0
1,50.1341,-326.0000,SAT,MZBER,0,0,6,-4.5,0
2,124.3276,-275.1935,LCV,MZBER,0,0,3,-2.5,0
3,50.1341,-326.0000,SAT,MZBER,0,0,3,-4.5,0
4,85.3905,-298.8632,XJB,MZBER,0,0,5,-4.5,0


In [82]:
data.shape

(2000, 9)

In [83]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x1      2000 non-null   float64
 1   x2      2000 non-null   float64
 2   x3      2000 non-null   object 
 3   x4      2000 non-null   object 
 4   x5      2000 non-null   int64  
 5   x6      2000 non-null   int64  
 6   x7      2000 non-null   int64  
 7   x8      2000 non-null   float64
 8   target  2000 non-null   int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 140.8+ KB


In [84]:
data.describe()

,x1,x2,x5,x6,x7,x8,target
count,2000.000000,2000.000000,2000.00000,2000.000000,2000.000000,2000.000000,2000.000000
mean,101.074885,-298.282145,0.01250,0.027000,3.440500,-5.343500,0.011000
std,22.670474,16.596490,0.11113,0.162124,0.972591,1.570108,0.104329
min,50.134100,-326.000000,0.00000,0.000000,3.000000,-7.500000,0.000000
25%,89.501675,-308.930400,0.00000,0.000000,3.000000,-6.500000,0.000000
50%,105.236100,-297.825600,0.00000,0.000000,3.000000,-5.500000,0.000000
75%,116.023175,-288.169025,0.00000,0.000000,3.000000,-4.500000,0.000000
max,150.134100,-226.000000,1.00000,1.000000,8.000000,-1.500000,1.000000


Se observa que las variables numéricas están en escalas diferentes.

In [85]:
data.isna().sum()

x1        0
x2        0
x3        0
x4        0
x5        0
x6        0
x7        0
x8        0
target    0
dtype: int64

No se encuentran NA en el dataframe.

In [86]:
data.nunique()

x1        1183
x2        1149
x3           4
x4           4
x5           2
x6           2
x7           5
x8           7
target       2
dtype: int64

In [87]:
data['target'].value_counts()

target
0    1978
1      22
Name: count, dtype: int64

In [95]:
print('Porcentaje de valores del target igual a 1: ', round(len(data[data['target'] == 1])/len(data) * 100,2), '%')

Porcentaje de valores del target igual a 1:  1.1 %


In [89]:
data['x3'].value_counts()

x3
LCV    926
SAT    715
XJB    280
QKP     79
Name: count, dtype: int64

In [90]:
data['x4'].value_counts()

x4
MZBER    1918
YEQA       41
PQKE       36
ZUQF        5
Name: count, dtype: int64

### Modelo utilizado

Debido a la urgencia y que solo se puede ocupar un modelo, se decide ocupar una regresión logística debido a que funciona bien con poca cantidad de datos y es rápida de utilizar. La regresión logística se utiliza con un solver 'newton-cg' debido a la poca cantidad de datos. Además, se utiliza OneHotEncoder para las variables categóricas y se le añade el balanceo de clases a la hora de ejecutar el algoritmo.

**Principales Resultados:** Utilizando regresión logística se obtiene una predicción con un F1-Score de 0.94 para los target 0, indicando que es un buen predictor para estos target, pero solo un F1-Score de 0.09 para los que son igual a 1, siendo un predictor poco preciso para estos target. Además, el modelo tiene valores de AUC de 0.90 en entrenamiento y de 0.71 en prueba, lo que indica que el modelo está capturando bien las diferentes clases en el set de entrenamiento, pero no lo hace de la misma forma al pasarle datos nuevos, esto puede indicar cierto sobreajuste. El modelo podría capturar mejor la diferencia de clases si se pudiera contar con una mayor cantidad de datos.

#### Preprocesamiento

In [91]:
# Se seleccionan las variables categóricas y numéricas
col_categoricas = ['x3', 'x4']
col_numericas = [col for col in data.columns if col not in col_categoricas]
col_numericas.remove('target')

# Se crea el preprocessor con OneHotEncoder para las variables categóricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', col_numericas),
        ('cat', OneHotEncoder(), col_categoricas) 
    ])

# Se crea el pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),                    
    ('classifier', LogisticRegression(class_weight='balanced', solver='newton-cg')) 
])

#### Entrenamiento y Prueba

In [92]:
# Dividir el conjunto de datos en entrenamiento y prueba
X = data.drop('target', axis=1)
y = data['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Entrenar el pipeline
pipeline.fit(X_train, y_train)

# Predecir sobre el conjunto de prueba
y_pred = pipeline.predict(X_test)

# Evaluar el modelo
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[533  60]
 [  4   3]]
              precision    recall  f1-score   support

           0       0.99      0.90      0.94       593
           1       0.05      0.43      0.09         7

    accuracy                           0.89       600
   macro avg       0.52      0.66      0.51       600
weighted avg       0.98      0.89      0.93       600



In [93]:
# Predecir las probabilidades en el conjunto de entrenamiento y prueba
y_train_pred_proba = pipeline.predict_proba(X_train)[:, 1]
y_test_pred_proba = pipeline.predict_proba(X_test)[:, 1]

# Calcular el AUC para el conjunto de entrenamiento
auc_train = roc_auc_score(y_train, y_train_pred_proba)
print(f"AUC en entrenamiento: {auc_train:.4f}")

# Calcular el AUC para el conjunto de prueba
auc_test = roc_auc_score(y_test, y_test_pred_proba)
print(f"AUC en prueba: {auc_test:.4f}")

AUC en entrenamiento: 0.9069
AUC en prueba: 0.7138
